# Система Сравнительного Анализа Методов Фрагментации Текста

## 1. Подготовка окружения и установка библиотек

In [1]:
# =============================================================================
# ЯЧЕЙКА 1: Установка необходимых библиотек
# =============================================================================

# Устанавливаем библиотеку PyMuPDF для работы с PDF (-q для тихого режима)
!pip install pymupdf -q
# Устанавливаем официальный клиент OpenAI API (совместим с Nebius Studio)
!pip install openai -q
# Устанавливаем NumPy для эффективных векторных операций и математики
!pip install numpy -q
# Устанавливаем tqdm для отображения прогресс-баров во время длительных операций
!pip install tqdm -q

# Выводим подтверждение успешной установки
print("✅ Все необходимые пакеты установлены!")
print("📦 Подготовлены: fitz, openai, numpy, tqdm")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.2 MB/s eta 0:00:00
✅ Все необходимые пакеты установлены!
📦 Подготовлены: fitz, openai, numpy, tqdm


In [2]:
# =============================================================================
# ЯЧЕЙКА 2: Импорт модулей
# =============================================================================

import fitz # Модуль 'fitz' (часть PyMuPDF) для чтения содержимого PDF
import os # Модуль для работы с переменными окружения, включая API-ключ
import json # Модуль для парсинга и работы с данными в формате JSON
import re # Модуль для операций с регулярными выражениями (для очистки текста)
from openai import OpenAI # Класс клиента для взаимодействия с API
from tqdm import tqdm # Класс для создания прогресс-баров для визуализации процесса
import numpy as math_ops # Импортируем NumPy под псевдонимом math_ops
from google.colab import files # Модуль для загрузки файлов в среду Colab

# Информируем пользователя о готовности
print("✅ Все модули успешно импортированы!")

✅ Все модули успешно импортированы!


## 2. Настройка API и загрузка документа

In [4]:
# =============================================================================
# ЯЧЕЙКА 3: Настройка клиента для работы с LLM API
# =============================================================================

# ⚠️ ВАЖНО: Замените 'ВАШ_СЕКРЕТНЫЙ_КЛЮЧ' на ваш действующий API-ключ Nebius Studio
# Ключ необходим для доступа к моделям эмбеддингов и генерации ответов
os.environ["OPENAI_API_KEY"] = "ВАШ_СЕКРЕТНЫЙ_КЛЮЧ"

# Инициализация клиента для API-взаимодействия
# Указываем базовый URL для Nebius Studio
llm_client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/", # Адрес конечной точки API
    api_key=os.getenv("OPENAI_API_KEY") # Получаем ключ из переменных окружения
)

# Проверка работоспособности подключения к сервису
print("🔑 API-клиент настроен и готов к работе!")

try:
    # Выполняем тестовый запрос на создание простого эмбеддинга
    test_vector_response = llm_client.embeddings.create(
        model="BAAI/bge-multilingual-gemma2", # Модель для создания тестового эмбеддинга
        input=["проверка связи"] # Текст для векторизации
    )
    # Выводим подтверждение и размерность полученного вектора
    print(f"🎯 Подключение успешно! Размерность вектора: {len(test_vector_response.data[0].embedding)}")
except Exception as err:
    # Обработка возможных ошибок при подключении или неверном ключе
    print(f"❌ Критическая ошибка при инициализации: {err}")
    print("💡 Пожалуйста, перепроверьте ваш API-ключ и интернет-соединение")

🔑 API-клиент настроен и готов к работе!
🎯 Подключение успешно! Размерность вектора: 3584


In [5]:
# =============================================================================
# ЯЧЕЙКА 4: Загрузка PDF-файла и извлечение из него текста
# =============================================================================

def read_pdf_content(file_path_name):
    """
    Функция для извлечения всего текстового содержимого из PDF-файла.
    Использует библиотеку fitz (PyMuPDF).
    """
    document_handle = fitz.open(file_path_name) # Открываем PDF-документ
    full_document_text = "" # Инициализируем пустую строку для текста
    for page_number in range(document_handle.page_count): # Проходим по всем страницам
        page = document_handle.load_page(page_number) # Загружаем текущую страницу
        full_document_text += page.get_text("text") + " " # Извлекаем текст и добавляем пробел
    document_handle.close() # Закрываем дескриптор файла после работы
    return full_document_text.strip() # Возвращаем очищенный от лишних пробелов текст

# Интерактивная загрузка файла пользователем
print("📂 Запуск виджета загрузки. Выберите ваш PDF-документ...")
uploaded_files = files.upload() # Вызов функции загрузки

# Получаем имя и путь загруженного файла
file_name = list(uploaded_files.keys())[0] # Берем имя первого (и единственного) загруженного файла

# Сохраняем содержимое файла локально
with open(file_name, 'wb') as f_out: # Открываем файл для бинарной записи
    f_out.write(uploaded_files[file_name]) # Записываем содержимое

print(f"✅ Файл '{file_name}' успешно сохранен в окружении Colab.")

# Попытка извлечения текста
try:
    document_text_content = read_pdf_content(file_name) # Вызываем функцию извлечения текста
    # Вывод статистики по извлеченному тексту
    print(f"📄 Успешно извлечено {len(document_text_content):,} символов текста")
    print(f"📝 Начало текста (первые 300 символов):\n{document_text_content[:300]}...")
    print(f"📈 Статистическая сводка:")
    total_words = len(document_text_content.split())
    print(f"   📏 Общее число символов: {len(document_text_content):,}")
    print(f"   📝 Количество слов (оценка): {total_words:,}")

except Exception as err_extract:
    # Сообщение об ошибке при извлечении
    print(f"❌ Не удалось извлечь текст из PDF: {err_extract}")
    # Заглушка, если произошла ошибка (чтобы не упал следующий код)
    document_text_content = "" # Обеспечиваем, что переменная объявлена

# Если произошла ошибка и текст пустой, останавливаем выполнение
if not document_text_content:
    raise Exception("❌ Критическая ошибка: Не удалось получить текст из документа. Проверьте файл.")

📂 Запуск виджета загрузки. Выберите ваш PDF-документ...


Saving instrukciya-samsung-galaxy-s23-ultra-6.pdf to instrukciya-samsung-galaxy-s23-ultra-6.pdf
✅ Файл 'instrukciya-samsung-galaxy-s23-ultra-6.pdf' успешно сохранен в окружении Colab.
📄 Успешно извлечено 274,210 символов текста
📝 Начало текста (первые 300 символов):
www.samsung.com
Russian. Rev.1.0
РУКОВОДСТВО 
ПОЛЬЗОВАТЕЛЯ
SM-S911B/DS
SM-S916B/DS
SM-S918B/DS
 2
Содержание
Начало работы
5	
Внешний вид устройства и функции
11	
Зарядка аккумулятора
16	
Карта nano-SIM и eSIM
18	
Включение и выключение устройства
19	
Начальная настройка
20	
Samsung account
21	
Пере...
📈 Статистическая сводка:
   📏 Общее число символов: 274,210
   📝 Количество слов (оценка): 35,715


## 3. Методы фрагментации (Chunking)

In [6]:
# =============================================================================
# ЯЧЕЙКА 5: Разбиение текста на фрагменты (чанки) фиксированной длины
# =============================================================================

def generate_fixed_length_fragments(text_data, chunk_size, overlap_size):
    """
    Создает список фрагментов текста заданной длины с заданным перекрытием.
    """
    fragment_list = [] # Список для хранения итоговых фрагментов
    text_length = len(text_data) # Общая длина исходного текста
    step_size = chunk_size - overlap_size # Размер шага для смещения (длина полезной части)

    # Проходим по тексту, начиная с индекса 0
    for i in range(0, text_length, step_size):
        # Вычисляем конечный индекс фрагмента
        end_index = i + chunk_size
        # Берем фрагмент от текущей позиции до end_index (или до конца текста)
        fragment = text_data[i:end_index]
        fragment_list.append(fragment) # Добавляем фрагмент в список
        # Если конец фрагмента достиг или превысил конец текста
        if end_index >= text_length:
            break # Завершаем цикл

    return fragment_list # Возвращаем список фрагментов

# Задаем параметры для тестирования
CHUNKS_128_SIZE = 128 # Размер чанка в символах (малый)
CHUNKS_256_SIZE = 256 # Размер чанка в символах (средний)
CHUNKS_512_SIZE = 512 # Размер чанка в символах (крупный)
OVERLAP_RATIO = 0.2 # Коэффициент перекрытия (20%)

# Рассчитываем размер перекрытия в символах для каждого размера чанка
overlap_128 = int(CHUNKS_128_SIZE * OVERLAP_RATIO)
overlap_256 = int(CHUNKS_256_SIZE * OVERLAP_RATIO)
overlap_512 = int(CHUNKS_512_SIZE * OVERLAP_RATIO)

# Словарь для хранения результатов всех методов чанкования
fragment_collections = {}

# Создание фрагментов для размера 128
fragments_128 = generate_fixed_length_fragments(document_text_content, CHUNKS_128_SIZE, overlap_128)
fragment_collections['Fixed_128'] = fragments_128
print(f"✅ Создано {len(fragments_128):,} фрагментов размером {CHUNKS_128_SIZE} с перекрытием {overlap_128} символов.")

# Создание фрагментов для размера 256
fragments_256 = generate_fixed_length_fragments(document_text_content, CHUNKS_256_SIZE, overlap_256)
fragment_collections['Fixed_256'] = fragments_256
print(f"✅ Создано {len(fragments_256):,} фрагментов размером {CHUNKS_256_SIZE} с перекрытием {overlap_256} символов.")

# Создание фрагментов для размера 512
fragments_512 = generate_fixed_length_fragments(document_text_content, CHUNKS_512_SIZE, overlap_512)
fragment_collections['Fixed_512'] = fragments_512
print(f"✅ Создано {len(fragments_512):,} фрагментов размером {CHUNKS_512_SIZE} с перекрытием {overlap_512} символов.")

# Выводим пример для наглядности
print(f"📝 Пример первого фрагмента (512 символов):'{fragments_512[0][:150]}...'" )

✅ Создано 2,662 фрагментов размером 128 с перекрытием 25 символов.
✅ Создано 1,338 фрагментов размером 256 с перекрытием 51 символов.
✅ Создано 669 фрагментов размером 512 с перекрытием 102 символов.
📝 Пример первого фрагмента (512 символов):'www.samsung.com
Russian. Rev.1.0
РУКОВОДСТВО 
ПОЛЬЗОВАТЕЛЯ
SM-S911B/DS
SM-S916B/DS
SM-S918B/DS
 2
Содержание
Начало работы
5	
Внешний вид устройства и...'


In [9]:
# =============================================================================
# ЯЧЕЙКА 6: Вспомогательные функции для Семантического чанкования
# =============================================================================

EMBEDDING_MODEL_NAME = "BAAI/bge-multilingual-gemma2" # Название модели для создания векторов
BATCH_SIZE = 128 # Размер батча для API-запросов (оптимизация скорости)

def get_text_units(input_text):
    """
    Разбивает текст на отдельные предложения (единицы текста).
    Использует регулярное выражение для поиска знаков препинания.
    """
    # Шаблон регулярного выражения: ищем точку, восклицательный/вопросительный знак,
    # за которым следуют пробел или конец строки (с учетом сокращений).
    units = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', input_text.strip())
    # Удаляем пустые строки и возвращаем список
    return [unit.strip() for unit in units if unit.strip()]


def generate_vector_batch(text_list, api_client):
    """
    Создает эмбеддинги (векторы) для списка текстовых строк, используя API.
    """
    try:
        # Отправляем запрос на создание эмбеддингов
        response = api_client.embeddings.create(
            model=EMBEDDING_MODEL_NAME, # Указываем модель для векторизации
            input=text_list # Список текстов для обработки
        )
        # Извлекаем сами векторы и преобразуем их в массив numpy
        return [math_ops.array(d.embedding) for d in response.data]
    except Exception as e:
        # В случае ошибки выводим предупреждение и возвращаем пустой список
        print(f"API Error during batch creation: {e}")
        return []

def calculate_vector_closeness(vector_a, vector_b):
    """
    Вычисляет косинусное сходство между двумя векторами NumPy.
    Значение от -1 (противоположное) до 1 (одинаковое направление).
    """
    # Норма векторов (длина)
    norm_a = math_ops.linalg.norm(vector_a)
    norm_b = math_ops.linalg.norm(vector_b)

    # Защита от деления на ноль (если вектор нулевой)
    if norm_a == 0 or norm_b == 0:
        return 0.0

    # Формула косинусного сходства: скалярное произведение / произведение норм
    similarity = math_ops.dot(vector_a, vector_b) / (norm_a * norm_b)
    return similarity

In [10]:
# =============================================================================
# ЯЧЕЙКА 7: Запуск Семантического чанкования
# =============================================================================

def segment_by_meaning(text_input, api_client, percentile_threshold=10):
    """
    Разбивает текст на семантические фрагменты, группируя предложения
    до момента значительного падения сходства между соседними предложениями.
    """
    sentences = get_text_units(text_input) # Шаг 1: Разбить текст на предложения
    if len(sentences) <= 1: # Если текста нет или мало
        return sentences # Возвращаем его целиком

    print(f"📊 Обнаружено {len(sentences)} предложений. Начинаем векторизацию...")

    sentence_vectors = [] # Список для хранения векторов предложений
    # Шаг 2: Создание векторов для всех предложений (с батчингом)
    for i in tqdm(range(0, len(sentences), BATCH_SIZE), desc="Векторизация предложений"):
        current_batch = sentences[i:i + BATCH_SIZE] # Выбираем текущий батч
        batch_vectors = generate_vector_batch(current_batch, api_client) # Генерируем векторы

        if batch_vectors: # Если векторы успешно получены
            sentence_vectors.extend(batch_vectors) # Добавляем их в общий список
        else:
            print(f"⚠️ Пропущена ошибка в батче {i//BATCH_SIZE + 1}. Продолжаем...")

    # Проверка, что количество векторов соответствует количеству предложений
    if len(sentence_vectors) != len(sentences):
        print("❌ ВНИМАНИЕ: Число векторов не соответствует числу предложений. Пропускаем семантическое чанкование.")
        return []

    print(f"✅ Создано {len(sentence_vectors)} векторов. Расчет сходства...")

    # Шаг 3: Вычисляем сходство между каждым соседним предложением
    vector_similarities = [] # Список для хранения значений сходства
    for i in range(len(sentence_vectors) - 1): # Проходим по парам векторов
        # Сравниваем вектор i и вектор i+1
        sim = calculate_vector_closeness(sentence_vectors[i], sentence_vectors[i + 1])
        vector_similarities.append(sim) # Добавляем результат

    # Шаг 4: Определяем порог для разрыва (наименьшее сходство)
    # Находим пороговое значение, например, 10-й процентиль (самые низкие 10% сходств)
    threshold_value = math_ops.percentile(vector_similarities, percentile_threshold)
    print(f"📉 Порог сходства (для разрыва): {threshold_value:.3f} (по {percentile_threshold}-му процентилю)")

    # Шаг 5: Формируем семантические фрагменты
    semantic_fragments = [] # Итоговый список семантических фрагментов
    current_fragment = "" # Временная строка для накопления текущего фрагмента

    for i, sentence in enumerate(sentences): # Проходим по всем предложениям
        current_fragment += sentence.strip() + " " # Добавляем предложение в текущий фрагмент

        # Если это не последнее предложение
        if i < len(sentences) - 1:
            # И если сходство с последующим предложением ниже порога
            if vector_similarities[i] < threshold_value:
                semantic_fragments.append(current_fragment.strip()) # Фиксируем готовый фрагмент
                current_fragment = "" # Начинаем новый фрагмент

    # Добавляем последний накопленный фрагмент (если он не пуст)
    if current_fragment.strip():
        semantic_fragments.append(current_fragment.strip())

    return semantic_fragments # Возвращаем результат

# Выполняем семантическое чанкование
semantic_chunks = segment_by_meaning(document_text_content, llm_client)
fragment_collections['Semantic'] = semantic_chunks

print(f"\n✅ Завершено! Создано {len(semantic_chunks)} семантических фрагментов.")

# Анализ результатов
chunk_lengths_semantic = [len(chunk) for chunk in semantic_chunks] # Длины семантических фрагментов
if chunk_lengths_semantic:
    print(f"📈 Статистика по семантическим фрагментам:")
    print(f"📏 Средняя длина: {math_ops.mean(chunk_lengths_semantic):.0f} символов")
    print(f"📐 Минимальная длина: {math_ops.min(chunk_lengths_semantic)} символов")
    print(f"📐 Максимальная длина: {math_ops.max(chunk_lengths_semantic)} символов")

print(f"\n📋 Сравнительная таблица методов:")
print("Тип фрагментации     Количество   Средняя длина (символов)")
print("-" * 50)
for frag_type, frags in fragment_collections.items():
    if frags:
        avg_len = math_ops.mean([len(f) for f in frags]) # Средняя длина фрагмента
        print(f"{str(frag_type):20} {len(frags):10,} {avg_len:23.0f}")
    else:
        print(f"{str(frag_type):20} {0:10,} {0:23.0f}")

📊 Обнаружено 2687 предложений. Начинаем векторизацию...


Векторизация предложений: 100%|██████████| 21/21 [00:24<00:00,  1.17s/it]

✅ Создано 2687 векторов. Расчет сходства...
📉 Порог сходства (для разрыва): 0.551 (по 10-му процентилю)

✅ Завершено! Создано 270 семантических фрагментов.
📈 Статистика по семантическим фрагментам:
📏 Средняя длина: 1014 символов
📐 Минимальная длина: 13 символов
📐 Максимальная длина: 11711 символов

📋 Сравнительная таблица методов:
Тип фрагментации     Количество   Средняя длина (символов)
--------------------------------------------------
Fixed_128                 2,662                     128
Fixed_256                 1,338                     256
Fixed_512                   669                     512
Semantic                    270                    1014


## 4. Векторизация фрагментов

In [11]:
# =============================================================================
# ЯЧЕЙКА 8: Создание эмбеддингов для всех фрагментов
# =============================================================================

def generate_all_fragment_vectors(api_client, fragments_list, method_name):
    """
    Создает векторные представления (эмбеддинги) для всех фрагментов
    с использованием батчинга и прогресс-бара.
    """
    all_vectors = [] # Список для хранения всех векторов

    if not fragments_list:
        print(f"⚠️ Для метода '{method_name}' нет фрагментов. Пропускаем векторизацию.")
        return []

    # Итерация по фрагментам с использованием батчей
    for i in tqdm(range(0, len(fragments_list), BATCH_SIZE), desc=f"Векторизация: {method_name}"):
        current_batch = fragments_list[i:i + BATCH_SIZE] # Выбираем батч фрагментов
        # Генерируем векторы для текущего батча, используя ранее определенную функцию
        batch_vectors = generate_vector_batch(current_batch, api_client)

        if batch_vectors: # Если получили векторы
            all_vectors.extend(batch_vectors) # Добавляем их к общему списку
        else:
            print(f"⚠️ Ошибка в батче {i//BATCH_SIZE + 1}. Пропускаем...")

    # Проверка на полное соответствие
    if len(all_vectors) != len(fragments_list):
        print(f"❌ ВНИМАНИЕ: Не удалось создать векторы для всех фрагментов метода '{method_name}'.")

    # Вывод результата
    print(f"✅ Создано {len(all_vectors)} векторов для метода '{method_name}'.")
    return all_vectors # Возвращаем список векторов

# Словарь для хранения векторов по типу фрагментации
fragment_vectors_map = {}

print("🧠 НАЧАЛО ВЕКТОРИЗАЦИИ ДЛЯ ВСЕХ МЕТОДОВ")
print("=" * 50)

# Проходим по всем коллекциям фрагментов и векторизуем их
for frag_type, fragments in fragment_collections.items():
    # Генерируем векторы и сохраняем их в map
    vectors = generate_all_fragment_vectors(llm_client, fragments, frag_type)
    fragment_vectors_map[frag_type] = vectors

print("\n🎉 Векторизация всех фрагментов завершена! Готовы к тестированию.")

🧠 НАЧАЛО ВЕКТОРИЗАЦИИ ДЛЯ ВСЕХ МЕТОДОВ


Векторизация: Fixed_128: 100%|██████████| 21/21 [00:26<00:00,  1.27s/it]


✅ Создано 2662 векторов для метода 'Fixed_128'.


Векторизация: Fixed_256: 100%|██████████| 11/11 [00:21<00:00,  1.94s/it]


✅ Создано 1338 векторов для метода 'Fixed_256'.


Векторизация: Fixed_512: 100%|██████████| 6/6 [00:17<00:00,  2.95s/it]


✅ Создано 669 векторов для метода 'Fixed_512'.


Векторизация: Semantic: 100%|██████████| 3/3 [00:10<00:00,  3.66s/it]

✅ Создано 270 векторов для метода 'Semantic'.

🎉 Векторизация всех фрагментов завершена! Готовы к тестированию.


## 5. Тестирование и Оценка (LLM-as-a-Judge)

Важный момент. Результаты работы подходов чанкинга и окончательного сравнения сильно будут зависеть от качества ваших тестовых вопросов и ответов на них

In [14]:
# =============================================================================
# ЯЧЕЙКА 9: Создание набора тестовых вопросов
# =============================================================================

print("❓ НАБОР ТЕСТОВЫХ ЗАПРОСОВ ДЛЯ ОЦЕНКИ")
print("=" * 45)
print("💡 Вам необходимо вручную создать несколько вопросов и эталонных ответов.")
print("   Эти пары используются для сравнения всех методов чанкования.")
print("   Рекомендуется 3-5 вопросов из вашего документа.\n")

# Запрос количества вопросов
try:
    # Пытаемся получить число вопросов. Если пользователь введет не число, будет ошибка
    num_test_queries = int(input("Сколько тестовых вопросов вы хотите создать? (рекомендуется 3-5): "))
except ValueError:
    print("⚠️ Введено не число. Устанавливаем 3 вопроса по умолчанию.")
    num_test_queries = 3

evaluation_queries = [] # Список для хранения вопросов и ответов

# Цикл для сбора вопросов
for i in range(min(num_test_queries, 10)): # Ограничение до 10 для безопасности
    print(f"\n{'='*25} ЗАПРОС #{i+1} {'='*25}")

    # Запрос самого вопроса
    query_text = input("💬 Вопрос пользователя (Например: 'Как получить список деталей?')\n→ ").strip()
    if not query_text:
        print("❌ Пропущен пустой вопрос. Завершаем сбор вопросов.")
        break

    # Запрос эталонного ответа (должен быть в документе)
    target_answer = input("✅ Правильный ответ\n→ ").strip()
    if not target_answer:
        print("❌ Пропущен пустой эталонный ответ. Завершаем сбор вопросов.")
        break

    # Сохранение вопроса в структуру данных
    evaluation_queries.append({
        "query": query_text,
        "reference_answer": target_answer
    })

    print(f"💾 Запрос #{i+1} добавлен в список для оценки.")

# Вывод созданных вопросов
print("\n📝 Сохраненные тестовые запросы:")
for idx, item in enumerate(evaluation_queries):
    print(f"   {idx+1}. Вопрос: {item['query'][:80]}...")
    print(f"      Эталон: {item['reference_answer'][:80]}...")

# Проверка на наличие вопросов
if not evaluation_queries:
    print("⚠️ Список вопросов пуст. Тестирование не будет выполнено.")

❓ НАБОР ТЕСТОВЫХ ЗАПРОСОВ ДЛЯ ОЦЕНКИ
💡 Вам необходимо вручную создать несколько вопросов и эталонных ответов.
   Эти пары используются для сравнения всех методов чанкования.
   Рекомендуется 3-5 вопросов из вашего документа.

Сколько тестовых вопросов вы хотите создать? (рекомендуется 3-5): 3

========================= ЗАПРОС #1 =========================
💬 Вопрос пользователя (Например: 'Как получить список деталей?')
→ Как разблокировать телефон?
✅ Правильный ответ
→ Нажать на боковую кнопку и провести по экрану пальцем
💾 Запрос #1 добавлен в список для оценки.

========================= ЗАПРОС #2 =========================
💬 Вопрос пользователя (Например: 'Как получить список деталей?')
→ Как перейти в настройки?
✅ Правильный ответ
→ Сделать свайп верхней шторки и нажать на шестерёнку
💾 Запрос #2 добавлен в список для оценки.

========================= ЗАПРОС #3 =========================
💬 Вопрос пользователя (Например: 'Как получить список деталей?')
→ Как открыть камеру?
✅ Правильны

In [21]:
# =============================================================================
# ЯЧЕЙКА 10: Функции для проведения тестирования и оценки RAG-системы
# =============================================================================

# Константы для моделей
GENERATOR_MODEL = "Qwen/Qwen3-32B" # Модель для генерации ответа (более быстрая)
JUDGE_MODEL = "openai/gpt-oss-120b" # Модель-судья для оценки качества (более мощная/точная)
VECTOR_SEARCH_TOP_K = 3 # Количество наиболее релевантных фрагментов для извлечения (Top-K)

# Константы для числовой оценки LLM-as-a-Judge
SCORE_COMPLETE = 1.0 # Полное соответствие / Полная релевантность
SCORE_PARTIAL = 0.5  # Частичное соответствие / Частичная релевантность
SCORE_NONE = 0.0     # Не соответствует / Неактуально

def retrieve_top_fragments(api_client, query_text, fragments_list, vectors_list, top_k=VECTOR_SEARCH_TOP_K):
    """
    Находит K наиболее релевантных фрагментов текста по заданному вопросу.
    """
    # Защита от пустых списков фрагментов/векторов
    if not fragments_list or not vectors_list or len(fragments_list) != len(vectors_list):
        return [], []

    # 1. Создаем вектор для вопроса
    query_vector = generate_vector_batch([query_text], api_client)
    if not query_vector:
        return [], [] # Если вектор не создан, возвращаем пустые списки

    query_emb = query_vector[0] # Получаем сам вектор вопроса

    # 2. Вычисляем косинусное сходство с векторами всех фрагментов
    closeness_scores = [] # Список для хранения пар (индекс, сходство)
    for i, fragment_vector in enumerate(vectors_list):
        # Используем функцию для косинусного сходства
        similarity = calculate_vector_closeness(query_emb, fragment_vector)
        closeness_scores.append((i, similarity)) # Сохраняем индекс и оценку

    # 3. Сортируем по сходству в порядке убывания и выбираем Top-K
    closeness_scores.sort(key=lambda x: x[1], reverse=True) # Сортировка по оценке
    top_k_indices = [item[0] for item in closeness_scores[:top_k]] # Индексы Top-K
    top_k_scores = [item[1] for item in closeness_scores[:top_k]] # Оценки Top-K

    # 4. Извлекаем сами фрагменты
    relevant_fragments = [fragments_list[i] for i in top_k_indices]

    return relevant_fragments, top_k_scores # Возвращаем фрагменты и их оценки сходства

def produce_llm_response(api_client, query_text, context_text):
    """
    Генерирует ответ LLM на основе вопроса и предоставленного контекста.
    """
    # Системный промпт для модели-генератора
    system_instruction = (
        "Ты — высокоточный ассистент, строго следующий предоставленному контексту. "
        "Ответь на вопрос пользователя, используя ИСКЛЮЧИТЕЛЬНО информацию из КОНТЕКСТА. "
        "Если в контексте нет ответа, честно скажи: 'В предоставленном контексте информация отсутствует.'"
    )

    # Контекст и запрос пользователя, объединенные в один промпт
    full_prompt = (
        f"КОНТЕКСТ ДЛЯ ОТВЕТА:\n---\n{context_text}\n---\n"
        f"ВОПРОС ПОЛЬЗОВАТЕЛЯ: {query_text}"
    )

    try:
        # Вызов API для генерации ответа
        response = api_client.chat.completions.create(
            model=GENERATOR_MODEL, # Модель для генерации
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": full_prompt}
            ],
            temperature=0.0 # Низкая температура для фактологической точности
        )
        # Извлекаем и возвращаем текст ответа
        return response.choices[0].message.content
    except Exception as e:
        print(f"❌ Ошибка генерации ответа: {e}")
        return "Внутренняя ошибка при генерации ответа."


def assess_response_quality(api_client, question, generated_answer, reference_answer):
    """
    Использует более мощную LLM (Judge) для оценки ответа по двум метрикам:
    Faithfulness (Верность фактам) и Relevancy (Актуальность).
    """
    # Шаблон промпта для оценки Верности фактам (Faithfulness)
    FAITHFULNESS_PROMPT = f"""
    Вы - эксперт-оценщик. Ваша задача - оценить ВЕРНОСТЬ ответа ИИ по отношению к ЭТАЛОННОМУ ОТВЕТУ.
    Верность (Faithfulness) измеряет, насколько факты в ответе ИИ совпадают с фактами в эталонном ответе.

    ЭТАЛОННЫЙ ОТВЕТ: {reference_answer}
    ОТВЕТ ИИ: {generated_answer}

    ИНСТРУКЦИИ:
    - Оцените СТРОГО используя только эти числовые значения:
     * {SCORE_COMPLETE} = Полностью соответствует, все факты верны
     * {SCORE_PARTIAL} = Частично соответствует, есть незначительные неточности/противоречия
     * {SCORE_NONE} = Не соответствует, серьезные галлюцинации или противоречия
    - Верните ТОЛЬКО числовую оценку ({SCORE_COMPLETE}, {SCORE_PARTIAL}, или {SCORE_NONE}) без каких-либо других символов или текста.
    """

    # Шаблон промпта для оценки Актуальности (Relevancy)
    RELEVANCY_PROMPT = f"""
    Вы - эксперт-оценщик. Ваша задача - оценить АКТУАЛЬНОСТЬ ответа ИИ по отношению к ВОПРОСУ ПОЛЬЗОВАТЕЛЯ.
    Актуальность (Relevancy) измеряет, насколько ответ ИИ отвечает именно на вопрос пользователя.

    ВОПРОС ПОЛЬЗОВАТЕЛЯ: {question}
    ОТВЕТ ИИ: {generated_answer}

    ИНСТРУКЦИИ:
    - Оцените СТРОГО используя только эти числовые значения:
     * {SCORE_COMPLETE} = Полностью актуален, напрямую отвечает на запрос
     * {SCORE_PARTIAL} = Частично актуален, отвечает неполно или задевает не все аспекты
     * {SCORE_NONE} = Неактуален, не отвечает на запрос
    - Верните ТОЛЬКО числовую оценку ({SCORE_COMPLETE}, {SCORE_PARTIAL}, или {SCORE_NONE}) без каких-либо других символов или текста.
    """

    # Функция для безопасного получения числовой оценки
    def get_score_from_llm(prompt_text):
        try:
            # Запрос к модели-судье
            res = api_client.chat.completions.create(
                model=JUDGE_MODEL, # Модель-судья
                messages=[{"role": "user", "content": prompt_text}],
                temperature=0.0
            )
            # Пытаемся преобразовать ответ в число с плавающей точкой
            score_str = res.choices[0].message.content.strip()
            # Упрощенная очистка для надежного парсинга числа
            score_str_clean = score_str.replace(',', '.') # Замена запятой на точку
            return float(score_str_clean)
        except Exception:
            # В случае ошибки парсинга или API возвращаем 0.0
            print(f"Ошибка парсинга или API при числовой оценке")
            return 0.0

    # Получаем оценки
    faithfulness_score = get_score_from_llm(FAITHFULNESS_PROMPT)
    relevancy_score = get_score_from_llm(RELEVANCY_PROMPT)

    return faithfulness_score, relevancy_score # Возвращаем обе метрики


def execute_evaluation_run(api_client, test_set, fragments, vectors, method_name):
    """
    Основная функция, запускающая полный цикл RAG-оценки для одного метода.
    """
    print(f"\n\n==================== ТЕСТИРОВАНИЕ МЕТОДА: {method_name} ====================")

    total_faithfulness = 0.0 # Сумма оценок верности
    total_relevancy = 0.0 # Сумма оценок актуальности
    processed_queries = 0 # Счетчик реально обработанных запросов (где были найдены фрагменты)

    # Проходим по каждому тестовому вопросу
    for i, item in enumerate(tqdm(test_set, desc="Оценка запросов")):
        query = item['query'] # Текущий вопрос пользователя
        reference = item['reference_answer'] # Эталонный ответ из документа

        print(f"\n📝 Запрос {i+1}/{len(test_set)}: {query}")

        # 1. Поиск наиболее близких фрагментов (Retrieval)
        relevant_fragments, scores = retrieve_top_fragments(api_client, query, fragments, vectors)

        if not relevant_fragments:
            print("❌ Фрагменты по вопросу не найдены. Пропускаем.")
            continue

        processed_queries += 1 # Увеличиваем счетчик

        print(f"🎯 Найдено {len(relevant_fragments)} релевантных фрагментов")
        print(f"📊 Максимальное сходство: {scores[0]:.3f}")

        # 2. Формирование контекста и генерация ответа (Generation)
        # Объединяем фрагменты в единый контекст для LLM
        context_data = "\n\n".join(relevant_fragments)
        generated_answer = produce_llm_response(api_client, query, context_data)

        # 3. Оценка ответа моделью-судьей (LLM-as-a-Judge)
        f_score, r_score = assess_response_quality(
            api_client, query, generated_answer, reference
        )

        total_faithfulness += f_score # Накопление верности
        total_relevancy += r_score # Накопление актуальности

        # Вывод детального результата
        print(f"🤖 Ответ ИИ: {generated_answer[:300]}...")
        print(f"✅ Эталон: {reference[:300]}...")
        print(f"⭐ Верность (Faithfulness): {f_score:.3f}")
        print(f"🎯 Актуальность (Relevancy): {r_score:.3f}")

    # 4. Вычисление итоговых средних метрик
    # Защита от деления на ноль, если не обработано ни одного запроса
    if processed_queries == 0:
        print("⚠️ Не было обработано ни одного запроса (не найдены фрагменты). Оценка: 0.0")
        avg_faithfulness, avg_relevancy, avg_composite_score = 0.0, 0.0, 0.0
    else:
        avg_faithfulness = total_faithfulness / processed_queries # Средняя верность
        avg_relevancy = total_relevancy / processed_queries # Средняя актуальность
        avg_composite_score = (avg_faithfulness + avg_relevancy) / 2 # Средняя метрика F&R

    # Вывод итогов
    print(f"\n\n📊 ИТОГОВЫЕ МЕТРИКИ ДЛЯ {method_name}: (N={processed_queries}/{len(test_set)})")
    print(f"   ⭐ Средняя Верность (Faithfulness): {avg_faithfulness:.3f}")
    print(f"   🎯 Средняя Актуальность (Relevancy): {avg_relevancy:.3f}")
    print(f"   ⭐+🎯 Средний Композитный балл: {avg_composite_score:.3f}")
    print(f"   📦 Общее число фрагментов: {len(fragments):,}")
    print(f"✅ Тестирование метода '{method_name}' завершено!")

    return {
        'method': method_name,
        'faithfulness': avg_faithfulness,
        'relevancy': avg_relevancy,
        'composite_score': avg_composite_score,
        'chunk_count': len(fragments)
    }

In [22]:
# =============================================================================
# ЯЧЕЙКА 11: Запуск тестирования и вывод общих результатов
# =============================================================================

if evaluation_queries: # Проверка, что есть вопросы для тестирования
    all_results = [] # Список для хранения результатов по всем методам

    # Проходим по всем методам фрагментации, которые были успешно созданы
    for method_key, fragments in fragment_collections.items():
        # Проверка, что для метода есть и фрагменты, и векторы
        if fragments and fragment_vectors_map.get(method_key):
            # Создаем более читабельное имя для отчета
            display_name = method_key.replace('_', ' ').replace('Fixed', 'Фиксированный')

            # Запуск оценки
            results = execute_evaluation_run(
                llm_client,
                evaluation_queries,
                fragments,
                fragment_vectors_map[method_key],
                display_name # Имя метода для вывода
            )
            all_results.append(results)
        else:
            print(f"⚠️ Пропущен метод '{method_key}': отсутствуют фрагменты или их векторы.")

    # =============================================================================
    # Сводная таблица результатов
    # =============================================================================

    if all_results: # Вывод только если есть результаты
        print("\n\n🏆 СВОДНЫЙ РЕЙТИНГ МЕТОДОВ ФРАГМЕНТАЦИИ")
        print("=" * 65)
        # Форматированный вывод заголовков таблицы
        print(f"{'Метод':25} {'Чанков':>8} {'Faithfulness':>15} {'Relevancy':>15} {'Композит':>15}")
        print("-" * 65)

        # Сортируем по композитному баллу (среднее F&R) для определения лучшего
        all_results.sort(key=lambda x: x['composite_score'], reverse=True)

        for res in all_results:
            # Форматированный вывод результатов для каждого метода
            print(
                f"{res['method']:25} "
                f"{res['chunk_count']:>8,} "
                f"{res['faithfulness']:>15.3f} "
                f"{res['relevancy']:>15.3f} "
                f"{res['composite_score']:>15.3f}"
            )

        print("-" * 65)
        # Выводим метод-победитель
        print(f"🥇 Победитель: {all_results[0]['method']} с композитным баллом {all_results[0]['composite_score']:.3f}")
    else:
        print("❌ Тестирование не запущено: не удалось подготовить ни один метод для оценки.")
else:
    # Сообщение, если не было вопросов для запуска
    print("❌ Тестирование не запущено, так как не было создано ни одного тестового вопроса.")



==================== ТЕСТИРОВАНИЕ МЕТОДА: Фиксированный 128 ====================


Оценка запросов:   0%|          | 0/3 [00:00<?, ?it/s]


📝 Запрос 1/3: Как разблокировать телефон?
🎯 Найдено 3 релевантных фрагментов
📊 Максимальное сходство: 0.854


Оценка запросов:  33%|███▎      | 1/3 [00:11<00:22, 11.46s/it]

🤖 Ответ ИИ: <think>
Хорошо, пользователь спрашивает, как разблокировать телефон. Нужно внимательно посмотреть предоставленный контекст. В контексте упоминаются пароль и PIN-код как способы разблокировки. Также есть упоминание о предварительно настроенных способах разблокировки, если лицо не распознано. 

Сначал...
✅ Эталон: Нажать на боковую кнопку и провести по экрану пальцем...
⭐ Верность (Faithfulness): 0.000
🎯 Актуальность (Relevancy): 0.500

📝 Запрос 2/3: Как перейти в настройки?
🎯 Найдено 3 релевантных фрагментов
📊 Максимальное сходство: 0.805


...

🤖 Ответ ИИ: <think>
Хорошо, пользователь спрашивает, как разблокировать телефон. Нужно найти информацию в предоставленном контексте. В контексте есть разделы о распознавании лица и отпечатков пальцев, а также о методах блокировки экрана.

Сначала проверю раздел "Разблокировка экрана по лицу". Там описаны шаги: ...
✅ Эталон: Нажать на боковую кнопку и провести по экрану пальцем...
⭐ Верность (Faithfulness): 0.000
🎯 Актуальность (Relevancy): 1.000

📝 Запрос 2/3: Как перейти в настройки?
🎯 Найдено 3 релевантных фрагментов
📊 Максимальное сходство: 0.721


Оценка запросов:  67%|██████▋   | 2/3 [00:55<00:25, 25.24s/it]

🤖 Ответ ИИ: <think>
Хорошо, пользователь спрашивает, как перейти в настройки. Нужно найти информацию в предоставленном контексте. В контексте много разделов, связанных с настройками, но конкретно о том, как открыть настройки, упоминаний нет. Проверю все разделы.

В разделе "Настройки" упоминается, что для измен...
✅ Эталон: Сделать свайп верхней шторки и нажать на шестерёнку...
⭐ Верность (Faithfulness): 0.000
🎯 Актуальность (Relevancy): 0.000

📝 Запрос 3/3: Как открыть камеру?
🎯 Найдено 3 релевантных фрагментов
📊 Максимальное сходство: 0.782


Оценка запросов: 100%|██████████| 3/3 [01:08<00:00, 22.92s/it]

🤖 Ответ ИИ: <think>
Хорошо, пользователь спрашивает, как открыть камеру. Нужно внимательно посмотреть контекст. В первом пункте приложения и функций 61 говорится, что приложение Камера можно открыть двукратным нажатием боковой клавиши или перетаскиванием значка влево на заблокированном экране. Также упоминается...
✅ Эталон: Открыть меню приложений и нажать на элемент Камера...
⭐ Верность (Faithfulness): 0.000
🎯 Актуальность (Relevancy): 1.000


📊 ИТОГОВЫЕ МЕТРИКИ ДЛЯ Semantic: (N=3/3)
   ⭐ Средняя Верность (Faithfulness): 0.000
   🎯 Средняя Актуальность (Relevancy): 0.667
   ⭐+🎯 Средний Композитный балл: 0.333
   📦 Общее число фрагментов: 270
✅ Тестирование метода 'Semantic' завершено!


🏆 СВОДНЫЙ РЕЙТИНГ МЕТОДОВ ФРАГМЕНТАЦИИ
Метод                       Чанков    Faithfulness       Relevancy        Композит
-----------------------------------------------------------------
Фиксированный 256            1,338           0.000           1.000           0.500
Фиксированный 128            2,6